# Preprocess ERA 5 data into monthly means
Builds monthly means for each year from available ERA5 data, including 10 m winds and mean sea level pressure

In [1]:
import warnings
warnings.filterwarnings('ignore')

## Set in paths

In [2]:
# data in
# ECMWF ERA5 base
base = '/uda/ERA5/Hourly_Data_On_Single_Levels/reanalysis/global/1hr-timestep/annual_file-range/'
# u-winds
u10_dir = base + 'Wind/u_10m/'
# v-winds
v10_dir = base + 'Wind/v_10m/'
# sea level pressure
slp_dir = base + 'Temperature_and_Pressure/P_mean_sea_level/'

## Preprocess algorithm

In [3]:
def preproc_ERA5_to_monthly(in_dir):
    # import pkgs
    from glob import glob
    import xarray as xr
    from dask.diagnostics import ProgressBar
    import numpy as np
    
    # Parse in_dir to get varaible
    var_dir = in_dir.split('/')[-2]
    
    # monthly means out
    out_dir = '/work/Marion.Alberty/data/ERA5_pp/' + var_dir + '/'
    
    # Get list of ERA5 files
    flist = glob(f'{in_dir}*.nc')
    
    # Read in data
    ds = xr.open_mfdataset(flist,use_cftime=True,engine='netcdf4').chunk({'time':24})
    
    # Time average and write data for each year
    with ProgressBar():
        for year in np.arange(ds.time.isel(time=0).dt.year.values,ds.time.isel(time=-1).dt.year.values+1):
            # Make time coord
            time = xr.cftime_range(start=str(year),periods=12,freq='MS')
            
            # Extract one year and get monthly average
            tmp = ds.sel(time=str(year)).groupby('time.month').mean().chunk(
                {'latitude':721,'longitude':1440,'month':12}).assign_coords(
                {"time": ("month", time)}).swap_dims({'month': 'time'})
            
            # Output to netcdf
            tmp.to_netcdf(out_dir + 'ERA5_monthly_' + var_dir + '_' + str(year) + '.nc')

## Set up DASK cluster
In dask labextension, create a +NEW cluster. Click the < > widget, scale it to 16 workers and let's go!

## Process and write variables

In [10]:
# u-winds
preproc_ERA5_to_monthly(u10_dir)

[########################################] | 100% Completed |  3min  1.4s
[########################################] | 100% Completed |  3min 10.5s
[########################################] | 100% Completed |  3min  2.4s
[########################################] | 100% Completed |  3min 26.0s
[########################################] | 100% Completed |  3min  2.4s
[########################################] | 100% Completed |  2min 49.4s
[########################################] | 100% Completed |  2min 49.5s
[########################################] | 100% Completed |  2min 18.2s
[########################################] | 100% Completed |  3min  9.1s
[########################################] | 100% Completed |  3min  2.2s
[########################################] | 100% Completed |  3min  6.4s
[########################################] | 100% Completed |  3min  3.9s
[########################################] | 100% Completed |  3min  0.9s
[#####################################

In [11]:
# v-winds
preproc_ERA5_to_monthly(v10_dir)

[########################################] | 100% Completed |  2min 16.3s
[########################################] | 100% Completed |  2min 33.0s
[########################################] | 100% Completed |  2min 20.2s
[########################################] | 100% Completed |  2min 44.1s
[########################################] | 100% Completed |  2min 45.3s
[########################################] | 100% Completed |  3min  3.5s
[########################################] | 100% Completed |  2min 54.7s
[########################################] | 100% Completed |  2min 59.1s
[########################################] | 100% Completed |  3min  0.9s
[########################################] | 100% Completed |  3min 23.6s
[########################################] | 100% Completed |  2min 45.0s
[########################################] | 100% Completed |  3min 10.7s
[########################################] | 100% Completed |  2min 27.4s
[#####################################

In [12]:
# sea level pressure
preproc_ERA5_to_monthly(slp_dir)

[########################################] | 100% Completed |  3min 13.2s
[########################################] | 100% Completed |  3min  0.3s
[########################################] | 100% Completed |  2min 28.5s
[########################################] | 100% Completed |  2min 34.7s
[########################################] | 100% Completed |  2min 35.8s
[########################################] | 100% Completed |  2min 13.7s
[########################################] | 100% Completed |  2min 16.2s
[########################################] | 100% Completed |  2min 23.5s
[########################################] | 100% Completed |  2min 30.2s
[########################################] | 100% Completed |  2min 27.6s
[########################################] | 100% Completed |  2min 20.5s
[########################################] | 100% Completed |  2min 46.3s
[########################################] | 100% Completed |  2min 46.0s
[#####################################